<a href="https://colab.research.google.com/github/Sridipta-Roy/Protein-Function-Prediction/blob/main/00_UniProt_Data_Fetcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive

# --- Mount drive and basic config ---
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/protein-multimodal')


Mounted at /content/drive


In [ ]:
PROJECT_ROOT = "/content/drive/MyDrive/protein-multimodal"
DATA_DIR = f"{PROJECT_ROOT}/data"
RAW_DIR = f"{DATA_DIR}/raw"

os.makedirs(RAW_DIR, exist_ok=True)

In [ ]:
!pip install -q requests tqdm

import json
import re
import time
import requests
from tqdm.notebook import tqdm
from collections import Counter

In [ ]:
# class UniProtDataFetcher:
#     """Fetches protein data from UniProt REST API with robust pagination"""

#     BASE_URL = "https://rest.uniprot.org/uniprotkb/search"

#     def __init__(self, max_proteins=10000):
#         self.max_proteins = max_proteins
#         self.proteins = []
#         self.accession_ids = set()  # Track unique accession IDs
#         self.duplicate_count = 0

#     def fetch_proteins(self, query):
#         """
#         Fetch high-quality proteins from UniProt with robust pagination

#         Query filters:
#         - reviewed:true      = Swiss-Prot (manually reviewed)
#         - organism_id:9606   = Human
#         """

#         print(f"🔍 Fetching proteins from UniProt...")
#         print(f"📋 Query: {query}")
#         print(f"🎯 Target: {self.max_proteins} proteins\n")

#         params = {
#             "query": query,
#             "format": "json",
#             "size": 500,  # Maximum batch size allowed by API
#             "fields": (
#                 "accession,id,protein_name,cc_function,sequence,length,"
#                 "organism_name,gene_names,go_id,ec"
#             ),
#         }

#         cursor = None
#         retry_count = 0
#         max_retries = 3

#         with tqdm(total=self.max_proteins, desc="Downloading", unit="proteins") as pbar:
#             while len(self.proteins) < self.max_proteins:
#                 # Use cursor-based pagination for reliability
#                 if cursor:
#                     params["cursor"] = cursor

#                 try:
#                     response = requests.get(self.BASE_URL, params=params, timeout=30)
#                     response.raise_for_status()
#                     data = response.json()

#                     # Check if we have results
#                     if "results" not in data or len(data["results"]) == 0:
#                         print("\n✓ No more results available from UniProt")
#                         break

#                     # Process this batch
#                     batch_added = 0
#                     batch_skipped = 0
#                     for entry in data["results"]:
#                         if len(self.proteins) >= self.max_proteins:
#                             break

#                         protein_data = self._parse_entry(entry)
#                         if protein_data:
#                             # Check for duplicate accession IDs
#                             acc_id = protein_data["accession"]
#                             if acc_id in self.accession_ids:
#                                 self.duplicate_count += 1
#                                 continue

#                             self.accession_ids.add(acc_id)
#                             self.proteins.append(protein_data)
#                             batch_added += 1
#                             pbar.update(1)
#                         else:
#                             batch_skipped += 1

#                     # Log batch statistics every 10 batches
#                     if len(self.proteins) % 5000 == 0 and len(self.proteins) > 0:
#                         print(f"\n  Progress: {len(self.proteins)} proteins collected")

#                     # Get cursor for next page
#                     cursor = data.get("nextCursor")

#                     # If no cursor, we've reached the end
#                     if not cursor:
#                         print("\n✓ Reached end of available results")
#                         break

#                     # Reset retry count on success
#                     retry_count = 0

#                     # Be nice to the API
#                     time.sleep(0.3)

#                 except requests.RequestException as e:
#                     retry_count += 1
#                     if retry_count >= max_retries:
#                         print(f"\n❌ Failed after {max_retries} retries: {e}")
#                         break

#                     print(f"\n⚠️  Error (retry {retry_count}/{max_retries}): {e}")
#                     time.sleep(2)  # Wait before retry

#         print(f"\n✓ Successfully fetched {len(self.proteins)} proteins")
#         print(f"  - Unique accession IDs: {len(self.accession_ids)}")
#         print(f"  - Duplicates found: {self.duplicate_count}")

#         return self.proteins

#     def _parse_entry(self, entry):
#         """Parse UniProt entry into structured format"""
#         try:
#             # Accession ID (primary unique identifier)
#             accession = entry.get("primaryAccession", "")

#             # Protein name
#             protein_name = (
#                 entry.get("proteinDescription", {})
#                 .get("recommendedName", {})
#                 .get("fullName", {})
#                 .get("value", "")
#             )

#             # Sequence
#             sequence = entry.get("sequence", {}).get("value", "")
#             seq_length = entry.get("sequence", {}).get("length", 0)

#             # Organism
#             organism = entry.get("organism", {}).get("scientificName", "")

#             # Gene name
#             genes = entry.get("genes", [])
#             gene_name = genes[0].get("geneName", {}).get("value", "") if genes else ""

#             # Function annotation from comments
#             function = ""
#             comments = entry.get("comments", [])
#             for comment in comments:
#                 if comment.get("commentType") == "FUNCTION":
#                     for text in comment.get("texts", []):
#                         function += text.get("value", "") + " "
#             function = function.strip()

#             # Quality filters
#             if not sequence:  # Must have sequence
#                 return None
#             if not function:  # Must have function annotation
#                 return None
#             if seq_length < 5:  # Very short sequences might be fragments
#                 return None

#             # Clean function text
#             function = self._clean_text(function)

#             # After cleaning, ensure we still have something meaningful
#             if len(function) < 20:  # Reduced from 50
#                 return None

#             # GO terms extraction
#             go_terms = []
#             for xref in entry.get("uniProtKBCrossReferences", []):
#                 if xref.get("database") == "GO":
#                     go_id = xref.get("id")
#                     go_term = None
#                     for prop in xref.get("properties", []):
#                         if prop.get("key") == "GoTerm":
#                             go_term = prop.get("value")
#                             break
#                     go_terms.append({
#                         "go_id": go_id,
#                         "go_term": go_term
#                     })

#             # EC numbers extraction
#             ec_numbers = []
#             ec_list = (
#                 entry.get("proteinDescription", {})
#                     .get("recommendedName", {})
#                     .get("ecNumbers", [])
#             )
#             for ec in ec_list:
#                 value = ec.get("value")
#                 if value:
#                     ec_numbers.append(value)

#             return {
#                 "accession": accession,
#                 "protein_name": protein_name,
#                 "gene_name": gene_name,
#                 "organism": organism,
#                 "sequence": sequence,
#                 "length": seq_length,
#                 "function": function,
#                 "go_terms": go_terms,
#                 "ec_numbers": ec_numbers,
#             }

#         except Exception as e:
#             # Skip malformed entries silently
#             return None

#     def _clean_text(self, text: str) -> str:
#         """Clean and normalize UniProt function text"""
#         # Remove references like {ECO:0000269|PubMed:12345}
#         text = re.sub(r"\{[^}]+\}", " ", text)

#         # Normalize spaces
#         text = re.sub(r"\s+", " ", text).strip()

#         # Remove weird characters but preserve basic punctuation
#         text = re.sub(r"[^\w\s\.,;:()\-/']", "", text)
#         text = re.sub(r"\s+", " ", text).strip()

#         return text

#     def save_raw_data(self, filepath):
#         """Save raw protein data to JSON"""
#         with open(filepath, "w") as f:
#             json.dump(self.proteins, f, indent=2)
#         print(f"✓ Saved raw data to {filepath}")

#     def verify_uniqueness(self):
#         """Verify that accession IDs are unique"""
#         accessions = [p["accession"] for p in self.proteins]
#         counter = Counter(accessions)
#         duplicates = {k: v for k, v in counter.items() if v > 1}

#         print(f"\n🔍 Uniqueness Verification:")
#         print(f"  - Total proteins: {len(self.proteins)}")
#         print(f"  - Unique accession IDs: {len(set(accessions))}")
#         print(f"  - Duplicates: {len(duplicates)}")

#         if duplicates:
#             print(f"  ⚠️  Found duplicate accession IDs:")
#             for acc, count in list(duplicates.items())[:5]:
#                 print(f"    - {acc}: {count} times")
#         else:
#             print(f"  ✓ All accession IDs are unique!")

#         return len(duplicates) == 0

In [ ]:
class UniProtDataFetcher:
    """Fetches protein data from UniProt REST API with robust pagination"""

    BASE_URL = "https://rest.uniprot.org/uniprotkb/search"

    def __init__(self, max_proteins=10000):
        self.max_proteins = max_proteins
        self.proteins = []
        self.accession_ids = set()  # Track unique accession IDs
        self.duplicate_count = 0

    def fetch_proteins(self, query="reviewed:true AND organism_id:9606"):
        """
        Fetch high-quality proteins from UniProt with robust pagination

        Query filters:
        - reviewed:true      = Swiss-Prot (manually reviewed)
        - organism_id:9606   = Human
        """

        print(f"🔍 Fetching proteins from UniProt...")
        print(f"📋 Query: {query}")
        print(f"🎯 Target: {self.max_proteins} proteins\n")

        params = {
            "query": query,
            "format": "json",
            "size": 500,  # Maximum batch size allowed by API
            "fields": (
                "accession,id,protein_name,cc_function,sequence,length,"
                "organism_name,gene_names,go_id,ec"
            ),
        }

        # Cursor in Link header, not JSON response
        cursor = None
        retry_count = 0
        max_retries = 3

        with tqdm(total=self.max_proteins, desc="Downloading", unit="proteins") as pbar:
            while len(self.proteins) < self.max_proteins:
                # Use cursor parameter if available
                if cursor:
                    params["cursor"] = cursor
                elif "cursor" in params:
                    # Remove cursor param for first request
                    del params["cursor"]

                try:
                    response = requests.get(self.BASE_URL, params=params, timeout=30)
                    response.raise_for_status()
                    data = response.json()

                    # Check if we have results
                    if "results" not in data or len(data["results"]) == 0:
                        print("\n✓ No more results available from UniProt")
                        break

                    # Process this batch
                    batch_added = 0
                    batch_skipped = 0
                    for entry in data["results"]:
                        if len(self.proteins) >= self.max_proteins:
                            break

                        protein_data = self._parse_entry(entry)
                        if protein_data:
                            # Check for duplicate accession IDs
                            acc_id = protein_data["accession"]
                            if acc_id in self.accession_ids:
                                self.duplicate_count += 1
                                continue

                            self.accession_ids.add(acc_id)
                            self.proteins.append(protein_data)
                            batch_added += 1
                            pbar.update(1)
                        else:
                            batch_skipped += 1

                    # Log batch statistics every 5000 proteins
                    if len(self.proteins) % 5000 == 0 and len(self.proteins) > 0:
                        print(f"\n  Progress: {len(self.proteins)} proteins collected")

                    # Extract cursor from Link header
                    cursor = self._extract_cursor_from_headers(response.headers)

                    # If no cursor in headers, we've reached the end
                    if not cursor:
                        print("\n✓ Reached end of available results (no Link header)")
                        break

                    # Reset retry count on success
                    retry_count = 0

                    time.sleep(0.3)

                except requests.RequestException as e:
                    retry_count += 1
                    if retry_count >= max_retries:
                        print(f"\n❌ Failed after {max_retries} retries: {e}")
                        break

                    print(f"\n⚠️  Error (retry {retry_count}/{max_retries}): {e}")
                    time.sleep(2)  # Wait before retry

        print(f"\n✓ Successfully fetched {len(self.proteins)} proteins")
        print(f"  - Unique accession IDs: {len(self.accession_ids)}")
        print(f"  - Duplicates found: {self.duplicate_count}")

        return self.proteins

    def _extract_cursor_from_headers(self, headers):
        """
        Extract cursor from Link header.

        UniProt returns cursor in Link header like:
        Link: <https://rest.uniprot.org/...?cursor=ABCD1234>; rel="next"
        """
        link_header = headers.get("Link", "")
        if not link_header:
            return None

        # Parse Link header to extract cursor
        # Format: <url?cursor=VALUE>; rel="next"
        import re
        match = re.search(r'cursor=([^&>]+)', link_header)
        if match:
            return match.group(1)

        return None

    def _parse_entry(self, entry):
        """Parse UniProt entry into structured format"""
        try:
            # Accession ID (primary unique identifier)
            accession = entry.get("primaryAccession", "")

            # Protein name
            protein_name = (
                entry.get("proteinDescription", {})
                .get("recommendedName", {})
                .get("fullName", {})
                .get("value", "")
            )

            # Sequence
            sequence = entry.get("sequence", {}).get("value", "")
            seq_length = entry.get("sequence", {}).get("length", 0)

            # Organism
            organism = entry.get("organism", {}).get("scientificName", "")

            # Gene name
            genes = entry.get("genes", [])
            gene_name = genes[0].get("geneName", {}).get("value", "") if genes else ""

            # Function annotation from comments
            function = ""
            comments = entry.get("comments", [])
            for comment in comments:
                if comment.get("commentType") == "FUNCTION":
                    for text in comment.get("texts", []):
                        function += text.get("value", "") + " "
            function = function.strip()

            # Quality filters
            if not sequence:  # Must have sequence
                return None
            if not function:  # Must have function annotation
                return None
            if seq_length < 30:  # Very short sequences might be fragments
                return None

            # Clean function text
            function = self._clean_text(function)

            # After cleaning, ensure we still have something meaningful
            if len(function) < 20:  # Reduced from 50
                return None

            # GO terms extraction
            go_terms = []
            for xref in entry.get("uniProtKBCrossReferences", []):
                if xref.get("database") == "GO":
                    go_id = xref.get("id")
                    go_term = None
                    for prop in xref.get("properties", []):
                        if prop.get("key") == "GoTerm":
                            go_term = prop.get("value")
                            break
                    go_terms.append({
                        "go_id": go_id,
                        "go_term": go_term
                    })

            # EC numbers extraction
            ec_numbers = []
            ec_list = (
                entry.get("proteinDescription", {})
                    .get("recommendedName", {})
                    .get("ecNumbers", [])
            )
            for ec in ec_list:
                value = ec.get("value")
                if value:
                    ec_numbers.append(value)

            return {
                "accession": accession,
                "protein_name": protein_name,
                "gene_name": gene_name,
                "organism": organism,
                "sequence": sequence,
                "length": seq_length,
                "function": function,
                "go_terms": go_terms,
                "ec_numbers": ec_numbers,
            }

        except Exception as e:
            # Skip malformed entries silently
            return None

    def _clean_text(self, text: str) -> str:
        """Clean and normalize UniProt function text"""
        # Remove references like {ECO:0000269|PubMed:12345}
        text = re.sub(r"\{[^}]+\}", " ", text)

        # Normalize spaces
        text = re.sub(r"\s+", " ", text).strip()

        # Keep ALL sentences instead of truncating to 3
        # (We want complete function descriptions for your MLLM training)

        # Remove weird characters but preserve basic punctuation
        text = re.sub(r"[^\w\s\.,;:()\-/']", "", text)
        text = re.sub(r"\s+", " ", text).strip()

        return text

    def save_raw_data(self, filepath):
        """Save raw protein data to JSON"""
        with open(filepath, "w") as f:
            json.dump(self.proteins, f, indent=2)
        print(f"✓ Saved raw data to {filepath}")

    def verify_uniqueness(self):
        """Verify that accession IDs are unique"""
        accessions = [p["accession"] for p in self.proteins]
        counter = Counter(accessions)
        duplicates = {k: v for k, v in counter.items() if v > 1}

        print(f"\n🔍 Uniqueness Verification:")
        print(f"  - Total proteins: {len(self.proteins)}")
        print(f"  - Unique accession IDs: {len(set(accessions))}")
        print(f"  - Duplicates: {len(duplicates)}")

        if duplicates:
            print(f"  ⚠️  Found duplicate accession IDs:")
            for acc, count in list(duplicates.items())[:5]:
                print(f"    - {acc}: {count} times")
        else:
            print(f"  ✓ All accession IDs are unique!")

        return len(duplicates) == 0

In [ ]:
MAX_PROTEINS = 10000

fetcher = UniProtDataFetcher(max_proteins=MAX_PROTEINS)

# Fetch proteins with cursor-based pagination
proteins = fetcher.fetch_proteins(
    query="reviewed:true AND organism_id:9606"
)

🔍 Fetching proteins from UniProt...
📋 Query: reviewed:true AND organism_id:9606
🎯 Target: 10000 proteins



Downloading:   0%|          | 0/10000 [00:00<?, ?proteins/s]


  Progress: 10000 proteins collected

✓ Successfully fetched 10000 proteins
  - Unique accession IDs: 10000
  - Duplicates found: 0


In [ ]:
#Verify Accession ID Uniqueness

is_unique = fetcher.verify_uniqueness()


🔍 Uniqueness Verification:
  - Total proteins: 10000
  - Unique accession IDs: 10000
  - Duplicates: 0
  ✓ All accession IDs are unique!


In [ ]:
raw_file = f"{RAW_DIR}/proteins_raw_10k.json"
fetcher.save_raw_data(raw_file)

✓ Saved raw data to /content/drive/MyDrive/protein-multimodal/data/raw/proteins_raw_10k.json


In [ ]:
if proteins:
    print(f"\nDataset Statistics:")
    print(f"  - Total proteins: {len(proteins)}")
    print(f"  - Average sequence length: {sum(p['length'] for p in proteins) / len(proteins):.0f}")
    print(f"  - Average function length (chars): {sum(len(p['function']) for p in proteins) / len(proteins):.0f}")
    print(f"  - Proteins with GO terms: {sum(1 for p in proteins if p['go_terms'])}")
    print(f"  - Proteins with EC numbers: {sum(1 for p in proteins if p['ec_numbers'])}")



Dataset Statistics:
  - Total proteins: 10000
  - Average sequence length: 623
  - Average function length (chars): 660
  - Proteins with GO terms: 9996
  - Proteins with EC numbers: 2796


In [ ]:
# Show sample protein
print(f"\nSample Protein:")
sample = proteins[0]
print(f"  - Accession: {sample['accession']}")
print(f"  - Name: {sample['protein_name']}")
print(f"  - Gene: {sample['gene_name']}")
print(f"  - Length: {sample['length']}")
print(f"  - Sequence: {sample['sequence']}")
print(f"  - Function: {sample['function']}")
print(f"  - GO terms: {len(sample['go_terms'])}")
print(f"  - EC numbers: {sample['ec_numbers']}")



Sample Protein:
  - Accession: A0A1B0GTW7
  - Name: Ciliated left-right organizer metallopeptidase
  - Gene: CIROP
  - Length: 788
  - Sequence: MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKSRSSSLTLPSSRDPQPLRIQSCYLGDHISDGAWDPEGEGMRGGSRALAAVREATQRIQAVLAVQGPLLLSRDPAQYCHAVWGDPDSPNYHRCSLLNPGYKGESCLGAKIPDTHLRGYALWPEQGPPQLVQPDGPGVQNTDFLLYVRVAHTSKCHQETVSLCCPGWSTAAQSQLTAALTSWAQRRGFVMLPRLCLKLLGSSNLPTLASQSIRITGPSVIAYAACCQLDSEDRPLAGTIVYCAQHLTSPSLSHSDIVMATLHELLHALGFSGQLFKKWRDCPSGFSVRENCSTRQLVTRQDEWGQLLLTTPAVSLSLAKHLGVSGASLGVPLEEEEGLLSSHWEARLLQGSLMTATFDGAQRTRLDPITLAAFKDSGWYQVNHSAAEELLWGQGSGPEFGLVTTCGTGSSDFFCTGSGLGCHYLHLDKGSCSSDPMLEGCRMYKPLANGSECWKKENGFPAGVDNPHGEIYHPQSRCFFANLTSQLLPGDKPRHPSLTPHLKEAELMGRCYLHQCTGRGAYKVQVEGSPWVPCLPGKVIQIPGYYGLLFCPRGRLCQTNEDINAVTSPPVSLSTPDPLFQLSLELAGPPGHSLGKEQQEGLAEAVLEALASKGGTGRCYFHGPSITTSLVFTVHMWKSPGCQGPSVATLHKALTLTLQKKPLEVYHGGANFTTQPSKLLVTSDHNPSMTHLRLSMGLCLMLLILVGVMGTTAYQKRATLPVRPSASYHSPELHSTRVPVRGIREV
  - Function: Putative metalloproteinase that plays a role in lef